# Exercise 3: Multiple walkers metadynamics

In the two previous two examples, the simulations we ran in parallel are indepdent of each other and can therefore be performed separately if wanted. This is different from methods such as multiple walkers metadynamics or replica exchange, where multiple simulations interact with each other and must be performed in parallel.

In this exercise, we will run 4 walkers of metadynamics for the NaCl system, from which we can estimate the free energy surface as a function of ion-pair distance. To compare with the results obtained from umbrella sampling in the previous exercise, we will perform the simulations in the NVT ensemble, which was also adopted in the previous tutorial on umbrella sampling. 

## 1. Running multiple walkers metadynamics

In [ ]:
%%bash
# Here we remove the output of the exercise if there is any.
dirs=(mpi_based)
for dir in "${dirs[@]}"; do
    if ls -d "$dir" >/dev/null 2>&1; then
        rm -rf "$dir"
    fi
done

Also, we define the PLUMED binary and a function that helps us run PLUMED commands.

In [ ]:
import subprocess
plumed_bin="singularity exec /ocean/projects/see220002p/shared/icomse_cpu.sif plumed"

def run_plumed_cmd(cmd,verbose=False):
    cmd_str="{} ".format(plumed_bin)+cmd
    print("PLUMED command: {}".format(cmd_str))
    if verbose:
        subprocess.run(cmd_str.split())
    else:
        subprocess.run(cmd_str.split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

To run 4 walkers of metadynamics that communicate with each other via MPI (i.e. MPI-based multiple walkers metadynamics), we will do the following:
- **Step 1**: Create folders named as `walker_0`, `walker_1`, `walker_2`, `walker_3`. 
- **Step 2**: In each folder, copy over the input files and generate a `tpr` file. 
- **Step 3**: In each folder, create a PLUMED input file `plumed.dat`. PLUMED input files across walkers should be the same.
- **Step 4**: Outside the folder, run the GROMACS `mdrun` command.

In [ ]:
%%bash
module load gromacs/2020.2-cpu openmpi/4.0.5-gcc10.2.0

mkdir mpi_based && cd mpi_based
for i in {0..3}
do
    mkdir walker_${i} && cd walker_${i}
    cp ../../../Inputs/NaCl/NaCl.top .
    cp ../../../Inputs/NaCl/configs/NaCl_${i}.gro .    # It is recommended to use different GRO files for different walkers
    cp ../../../Inputs/NaCl/MD-NVT.mdp .         # The mdp file for running NVT simulations
    mpirun -np 1 gmx_mpi grompp -f MD-NVT.mdp -c NaCl_${i}.gro -p NaCl.top -o multi_metad.tpr
    cd ../
done

In [ ]:
%%bash
cd mpi_based
for i in {0..3}
do 
cd walker_${i}
echo '# Distance between Na and Cl atoms
dist: DISTANCE ATOMS=322,323

uwall: UPPER_WALLS ...
   ARG=dist 
   AT=0.6
   KAPPA=4000.0 
...

METAD ...
  LABEL=mtd
  ARG=dist
  PACE=500
  SIGMA=0.02 
  HEIGHT=1.25
  BIASFACTOR=5
  GRID_MIN=0.0
  GRID_MAX=1.0
  CALC_RCT
  FILE=hills.data
  WALKERS_DIR=../
  WALKERS_MPI
... METAD

PRINT ARG=dist,mtd.*,uwall.* FILE=colvar.data STRIDE=250
' > plumed.dat
cd ../
done

In [ ]:
%%time
%%bash
module load gromacs/2020.2-cpu openmpi/4.0.5-gcc10.2.0

cd mpi_based
mpirun -np 4 gmx_mpi mdrun -s multi_metad.tpr -plumed plumed.dat -multidir walker_{0..3} -ntomp 1

## 2. Data analysis

As a result, `colvar.0.dat`, `colvar.1.dat`, `colvar.2.dat` and `colvar.3.dat` were generated in foldes `walker_0`, `walker_1`, `walker_2`, and `walker_3`, respectively. 

In [ ]:
%%bash
tree

As you learned yesterday, each of the `color.*.data` files kept track of the collective variable for each walker, from which we can visualize below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

# Just some settings for plotting
rc('font', **{
    'family': 'sans-serif',
    'sans-serif': ['DejaVu Sans'],
    'size': 10,
})
# Set the font used for MathJax - more on thiprint(images)
rc('mathtext', **{'default': 'regular'})
plt.rc('font', family='serif')

In [ ]:
plt.figure()
for i in range(4):
    dist = np.transpose(np.loadtxt(f'mpi_based/walker_{i}/colvar.{i}.data'))[1]
    plt.plot(np.arange(len(dist)) * 0.5, dist, alpha=0.7, linewidth=0.8, label=f'Walker {i}')
    plt.xlabel('Time (ps)')
    plt.ylabel('Ion-pair distance (nm)')
plt.legend(ncols=2)
plt.grid()

On the other hand, there will be only one `HILLS` output, which as specified in our PLUMED input file, is one level above the `walkers_*` directory and named as `hills.data`. From this file, we can rescontruct the free energy surface as a function of ion-pair distance using `sum_hills`.

In [ ]:
run_plumed_cmd('sum_hills --hills mpi_based/hills.data --outfile mpi_based/fes.dat --mintozero', verbose=True)

In [ ]:
fes_MetaD = np.transpose(np.loadtxt('mpi_based/fes.dat', comments=['@', '#']))
fes_US = np.transpose(np.loadtxt('../Exercise_2/fes.dat', comments=['@', '#']))
kT = 300 * 1.380649E-23 * 6.02214076E23 / 1000   # 1 kT in kJ/mol
plt.plot(fes_MetaD[0], fes_MetaD[1] / kT, label='Metadynamics')
plt.plot(fes_US[0], fes_US[1] - min(fes_US[1]), label='Umbrella sampling')
plt.fill_between(fes_US[0], fes_US[1] - fes_US[2], fes_US[1] + fes_US[2], color='lightgreen')
plt.xlabel('Ion-pair distance (nm)')
plt.ylabel('Free energy (kT)')
plt.legend()
plt.grid()

## 3. Reference(s)
- The paper that proposed multiple walkers metadynamics: [Raiteri, Paolo, et al. "Efficient reconstruction of complex free energy landscapes by multiple walkers metadynamics." The journal of physical chemistry B 110.8 (2006): 3533-3539.](https://pubs.acs.org/doi/10.1021/jp054359r)

## Takeaways

- Using multiple walkers metadynamics, we are able to get a free energy surface consistent with the one obtained from metadynamics we performed on Tuesday. 
- Multiple walkers metadynamics accelerate the exploration of the CV space is suitable espeically when multiple CVs are used. 